<a href="https://colab.research.google.com/github/brunovianna/krita-colab-notebook/blob/main/comfyui_colab_krita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [7]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

# @markdown If you have space to store the models in google drive, the start time will be much shorter after the first installation

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
# USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = True
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

#@title Choice of models to download

DOWNLOAD_PARAMS = ""
# @markdown ---
# @markdown #### Choice of models to download
# @markdown (uses this script: https://github.com/Acly/krita-ai-diffusion/blob/main/scripts/download_models.py )

# @markdown The bare *minimum*, *recommended* or *all* models. *All* takes a lot of time to start and disk space
MINIMAL = False  #@param {type:"boolean"}
RECOMMENDED = True # @param {"type":"boolean","placeholder":"False"}
ALL = False  #@param {type:"boolean"}

# @markdown All needed to run SD1.5, SDXL, or Flux. Checkpoints not included.
SD15 = False  #@param {type:"boolean"}
SDXL = True  #@param {type:"boolean"}
FLUX = False  #@param {type:"boolean"}

# @markdown All checkpoints for selected workloads
CHECKPOINTS = False  #@param {type:"boolean"}

# @markdown Upscalers
UPSCALERS = False  #@param {type:"boolean"}

# @markdown Prefetch:  models which would be automatically downloaded on first use
PREFETCH = False  #@param {type:"boolean"}



DOWNLOAD_PARAMS += " --minimal" if MINIMAL else ""
DOWNLOAD_PARAMS += " --recommended" if RECOMMENDED else ""
DOWNLOAD_PARAMS += " --all" if ALL else ""
DOWNLOAD_PARAMS += " --sd15" if SD15 else ""
DOWNLOAD_PARAMS += " --sdxl" if SDXL else ""
DOWNLOAD_PARAMS += " --flux" if FLUX else ""
DOWNLOAD_PARAMS += " --checkpoints" if CHECKPOINTS else ""
DOWNLOAD_PARAMS += " --upscalers" if UPSCALERS else ""
DOWNLOAD_PARAMS += " --prefetch" if PREFETCH else ""


CHECKPOINT_LIST = ""

# @markdown Individual choice of models SD1.5
SERENITY = True  #@param {type:"boolean"}
DREAMSHAPER = False  #@param {type:"boolean"}
FLAT2DANIMERGE = False #@param {type:"boolean"}

# @markdown Individual choice of models SDXL
REALVIS = False  #@param {type:"boolean"}
ZAVYCHROMA = True  #@param {type:"boolean"}
PIXELWAVE = True #@param {type:"boolean"}

# @markdown Individual choice of models FLUX
DEV = False  #@param {type:"boolean"}
SCHNELL = False  #@param {type:"boolean"}


CHECKPOINT_LIST += " --checkpoint serenity" if SERENITY else ""
CHECKPOINT_LIST += " --checkpoint dreamshaper" if DREAMSHAPER else ""
CHECKPOINT_LIST += " --checkpoint flat2d-animerge" if FLAT2DANIMERGE else ""
CHECKPOINT_LIST += " --checkpoint realvis" if REALVIS else ""
CHECKPOINT_LIST += " --checkpoint zavychroma" if ZAVYCHROMA else ""
CHECKPOINT_LIST += " --checkpoint pixelwave" if PIXELWAVE else ""
CHECKPOINT_LIST += " --checkpoint flux-dev" if DEV else ""
CHECKPOINT_LIST += " --checkpoint flux-schnell" if SCHNELL else ""


print (DOWNLOAD_PARAMS)
print (CHECKPOINT_LIST)

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/node_db/tutorial/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/tutorial/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies


 --recommended --sdxl
 --checkpoint serenity --checkpoint zavychroma --checkpoint pixelwave
Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
Updating d170292..e1dec3c
Fast-forward
 api_server/services/terminal_service.py                      |  6 ++--
 app/custom_node_manager.py                                   | 34 +++++++++++++++++++++
 app/model_manager.py                                         |  2 +-
 comfy/hooks.py                                               | 34 ++++++++++-----------
 comfy/model_patcher.py                                       | 30 +++++++++---------
 comfy/patcher_extension.py                                   |  6 ++--
 comfy_execution/graph.py                                     |  2 +-
 comfy_extras/nodes_hooks.py                                  | 52 ++++++++++++++++-----

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [8]:
# Checkpoints

### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/

# SDXL ReVision
#!wget -c https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -P ./models/clip_vision/

# SD1.5
#!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

# SD2
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

# Some SD1.5 anime style
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P ./models/checkpoints/

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
#!wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors -P ./models/checkpoints/


# unCLIP models
#!wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -P ./models/checkpoints/


# VAE
#!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P ./models/vae/
#!wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P ./models/vae/


# Loras
#!wget -c https://civitai.com/api/download/models/10350 -O ./models/loras/theovercomer8sContrastFix_sd21768.safetensors #theovercomer8sContrastFix SD2.x 768-v
#!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/theovercomer8sContrastFix_sd15.safetensors #theovercomer8sContrastFix SD1.x
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P ./models/loras/ #SDXL offset noise lora


# T2I-Adapter
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

# T2I Styles Model
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

# CLIPVision model (needed for styles model)
#!wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O ./models/clip_vision/clip_vit14.bin


# ControlNet
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/

# ControlNet SDXL
#!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -P ./models/controlnet/


# model downloader node
!cd custom_nodes && git clone https://github.com/ciri/model-downloader-comfyui.git

# Krita models and nodes

# Controlnet Preprocessor nodes by Fannovel16
!cd custom_nodes && git clone https://github.com/Fannovel16/comfyui_controlnet_aux; cd comfyui_controlnet_aux && pip install -r requirements.txt

# ip-adapter
!cd custom_nodes && git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git ;

# external tooling nodes
!cd custom_nodes && git clone https://github.com/Acly/comfyui-tooling-nodes.git

# inpaint
!cd custom_nodes && git clone https://github.com/Acly/comfyui-inpaint-nodes.git

# from https://github.com/Acly/krita-ai-diffusion/wiki/ComfyUI-Setup

!python -m pip install aiohttp tqdm websockets
#!cd /content/ComfyUI/krita-ai-diffusion/ && python scripts/download_models.py ../ --sd15 --sdxl --upscalers -- checkpoint serenity --checkpoint dreamshaper

if OPTIONS['USE_GOOGLE_DRIVE']:
    #change to content since it won't work in google drive
    !cd /content && git clone https://github.com/Acly/krita-ai-diffusion.git --recurse-submodules
    !cd /content/krita-ai-diffusion/ && python scripts/download_models.py /content/drive/MyDrive/ComfyUI $DOWNLOAD_PARAMS $CHECKPOINT_LIST
else:
    !git clone https://github.com/Acly/krita-ai-diffusion.git --recurse-submodules
    !cd krita-ai-diffusion/ && python scripts/download_models.py ../ $DOWNLOAD_PARAMS $CHECKPOINT_LIST



#!wget -c https://github.com/Acly/krita-ai-diffusion/raw/refs/heads/main/scripts/download_models.py

#!rm ./download_models.py



# clip vision
#!mkdir ./models/clipvision
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -P ./models/clipvision/clip-vision_vit-h.safetensors

# upscale
#!wget -c https://huggingface.co/gemasai/4x_NMKD-Superscale-SP_178000_G/resolve/main/4x_NMKD-Superscale-SP_178000_G.pth -P ./models/upscale_models/
#!wget -c https://huggingface.co/Acly/Omni-SR/resolve/main/OmniSR_X2_DIV2K.safetensors -P ./models/upscale_models/
#!wget -c https://huggingface.co/Acly/Omni-SR/resolve/main/OmniSR_X3_DIV2K.safetensors -P ./models/upscale_models/
#!wget -c https://huggingface.co/Acly/Omni-SR/resolve/main/OmniSR_X4_DIV2K.safetensors -P ./models/upscale_models/
#!wget -c https://huggingface.co/Acly/MAT/resolve/main/MAT_Places512_G_fp16.safetensors -P ./models/inpaint

# sd 1.5
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_lora_rank128_v11f1e_sd15_tile_fp16.safetensors -P ./models/controlnet
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -P ./models/ipadapter
#!wget -c https://huggingface.co/ByteDance/Hyper-SD/resolve/main/Hyper-SD15-8steps-CFG-lora.safetensors -P ./models/loras

# sdxl
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -P ./models/ipadapter
#!wget -c https://huggingface.co/ByteDance/Hyper-SD/resolve/main/Hyper-SDXL-8steps-CFG-lora.safetensors -P ./models/loras
#!wget -c https://huggingface.co/lllyasviel/fooocus_inpaint/resolve/main/fooocus_inpaint_head.pth -P ./models/inpaint
#!wget -c https://huggingface.co/lllyasviel/fooocus_inpaint/resolve/main/inpaint_v26.fooocus.patch -P ./models/inpaint

#checkpoints
#!wget -c https://huggingface.co/lllyasviel/fav_models/resolve/main/fav/realisticVisionV51_v51VAE.safetensors -P ./models/checkpoints/


Cloning into 'model-downloader-comfyui'...
fatal: could not read Username for 'https://github.com': No such device or address
fatal: destination path 'comfyui_controlnet_aux' already exists and is not an empty directory.
fatal: destination path 'ComfyUI_IPAdapter_plus' already exists and is not an empty directory.
fatal: destination path 'comfyui-tooling-nodes' already exists and is not an empty directory.
fatal: destination path 'comfyui-inpaint-nodes' already exists and is not an empty directory.
fatal: destination path 'krita-ai-diffusion' already exists and is not an empty directory.
Generative AI for Krita - Model download - v1.29.1
CLIP Vision model: found - skipping
ControlNet Universal (XL): 100% 2.34G/2.34G [00:59<00:00, 42.4MB/s]
Fooocus Inpaint: 100% 51.4k/51.4k [00:00<00:00, 5.41MB/s]
Fooocus Inpaint: 100% 1.23G/1.23G [00:09<00:00, 140MB/s]
HAT Super-Resolution (quality): 100% 81.2M/81.2M [00:02<00:00, 31.3MB/s]
Hyper-SD LoRA (SDXL): 100% 751M/751M [00:20<00:00, 38.3MB/s]
I

### Run ComfyUI with cloudflared (Recommended Way)




In [9]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2024-12-28 11:33:44--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2024.12.2/cloudflared-linux-amd64.deb [following]
--2024-12-28 11:33:44--  https://github.com/cloudflare/cloudflared/releases/download/2024.12.2/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/8f327838-88be-4fbe-822d-7ec8224c629c?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241228%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241228T113344Z&X-Amz-Expires=300&X-Amz-Signature=3702c0b44bf2b17c3da8bae1d1515be22ba2d78045efc8df8bb1e0e257

### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 4s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋npm notice
npm notice New major version of npm available! 10.8.2 -> 11.0.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.0.0
npm notice To update run: npm install -g npm@11.0.0
npm notice
⠋[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2024-12-27 14:16:20.241557
** Platform: Linux
** Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** Log path: /content/ComfyUI/comfyui.log

Prestartup times for custom nodes:
   3.9 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Total VRAM 15102 MB, total RAM 12979 MB
pytorch version: 2.5.1+cu121
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using pytorch attention
[Prompt Server] web root: /content/ComfyUI/web
### Lo

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
import time
import socket
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server